# Text sentiment analysis deployment 

In [13]:
import pickle
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
# Loading the vectoriser
file = open('vectoriser.pickle', 'rb')
vectoriser = pickle.load(file)
file.close()
# Loading the LR Model
file = open('Sentiment.pickle', 'rb')
LRmodel = pickle.load(file)
file.close()

In [14]:
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

In [15]:
def preprocess(textdata):
    processedText = []
    
    # Creating Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer() #converting words to their base or dictionary form
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)" #to match URLs in the text
    userPattern       = '@[^\s]+' #to identify and potentially remove user mentions from the text
    alphaPattern      = "[^a-zA-Z0-9]" #to remove non-alphanumeric characters from the text.
    sequencePattern   = r"(.)\1\1+" #to identify sequences of repeated characters (3 or more repetitions) in the text
    seqReplacePattern = r"\1\1" #to replace sequences of repeated characters with just two occurrences of the same character.
    
    for tweet in textdata:
        tweet = tweet.lower()
        tweet = re.sub(urlPattern,' URL',tweet)
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        tweet = re.sub(userPattern,' USER', tweet)
        
        tweet = re.sub(alphaPattern, " ", tweet)
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            if len(word)>1:
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [29]:
text =input('Enter a tweet:')
text=[text]
print()
print()
# Predict the sentiment
textdata = vectoriser.transform(preprocess(text))
sentiment = LRmodel.predict(textdata)
    
# Make a list of text with sentiment.
data = []
for text, pred in zip(text, sentiment):
    data.append((text,pred))
        
# Convert the list into a Pandas DataFrame.
df = pd.DataFrame(data, columns = ['text','sentiment'])
df = df.replace([0,1], ["Negative","Positive"])
print(df)

Enter a tweet:I hate twitter


             text sentiment
0  I hate twitter  Negative
